In [4]:
import keras

Using TensorFlow backend.


In [5]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

In [6]:
w2v_file = "../../Lab_sems/glove.6B.300d.txt"
def get_w2v(file_name):
    w2v_dict = {}
    f = open(file_name).readlines()
    for line in f:
        ls = line.split()
        word = ls[0]
        coefs = asarray(ls[1:], dtype='float32')
        w2v_dict[word] = coefs 
    return w2v_dict

In [7]:
w2v_dict = get_w2v(w2v_file)

In [26]:
def get_sents(filename):
    f = open(filename).readlines()
    sents = []
    tags = []
    sent = []
    tags_ = []
    tag_set = set()
    max_len = 0
    for line in f[1:]:
        if line == "\n":
            continue
        if "SAMPLE_START" in line or "[SEP]" in line:
            sents.append(sent)
            tags.append(tags_)
            if len(sent)> max_len:
                max_len = len(sent)
            sent = []
            tags_ = []
        else:
            ls = line.split()
            sent.append(ls[0])
            tags_.append(ls[-1])
            tag_set.add(ls[-1])
    if len(sent) > 0:
        sents.append(sent)
        tags.append(tags_)
    return sents,tags,tag_set,max_len
def arr_to_str(arr):
    return " ".join([x for x in arr])
def idx_to_arr(idx_arr,vocab):
    arr = []
    for idx in idx_arr:
        arr.append(vocab[idx-1])
    return arr
get_sents2 = lambda sents : [arr_to_str(sent) for sent in sents]

In [9]:
train_file = "train.txt"
dev_file = "dev.txt"
test_file = "test.txt"
train_sents,train_tags,train_tag_set, train_max_len = get_sents(train_file)
dev_sents,dev_tags,dev_tag_set, dev_max_len = get_sents(dev_file)
#dev_sents2 = get_sents2(dev_sents)
train_max_len = max(train_max_len,dev_max_len)
for sent,tag in zip(dev_sents,dev_tags):
    train_sents.append(sent)
    train_tags.append(tag)
    
train_tag_set.union(dev_tag_set)
vocab = set()


In [10]:
train_tag_set

{'B-etime',
 'B-fname',
 'B-loc',
 'B-organizer',
 'B-participant',
 'B-place',
 'B-target',
 'B-trigger',
 'I-etime',
 'I-fname',
 'I-loc',
 'I-organizer',
 'I-participant',
 'I-place',
 'I-target',
 'I-trigger',
 'O'}

In [11]:
train_max_len

107

In [12]:
for sent in train_sents:
    for word in sent:
        vocab.add(word)
vocab = list(vocab)
vocab.append("ENDPAD")

In [13]:
word2idx = {w: i + 1 for i, w in enumerate(vocab)}
tag2idx = {t: i for i, t in enumerate(train_tag_set)}

In [34]:
import string
string.alphabet

AttributeError: module 'string' has no attribute 'alphabet'

In [14]:
X = [[word2idx[x] for x in sent] for sent in train_sents]
Y = [[tag2idx[x]  for x in tag] for tag in train_tags]

In [32]:
for w,x,y in zip(X[0],train_tags[0],Y[0]):
    print(vocab[w-1],x,list(train_tag_set)[y])

KRISHNAGIRI O O
/ O O
DHARMAPURI O O
: O O
Water O O
supply O O
disrupted O O
, O O
villagers B-participant B-participant
block B-trigger B-trigger
road B-loc B-loc
September O O
04 O O
, O O
2014 O O
00:00 O O
IST O O
Officials O O
promise O O
to O O
supply O O
water O O
in O O
tankers O O
Women B-participant B-participant
took B-trigger B-trigger
to I-trigger I-trigger
streets I-trigger I-trigger
carrying O O
pots O O
in O O
protest O O
against O O
erratic O O
supply O O
of O O
drinking O O
water O O
at O O
Kelamagalam B-place B-place
in O O
Udhanapalli B-place B-place
on B-etime B-etime
Wednesday I-etime I-etime
. O O


In [27]:
idx_to_arr(X[0],vocab)

['KRISHNAGIRI',
 '/',
 'DHARMAPURI',
 ':',
 'Water',
 'supply',
 'disrupted',
 ',',
 'villagers',
 'block',
 'road',
 'September',
 '04',
 ',',
 '2014',
 '00:00',
 'IST',
 'Officials',
 'promise',
 'to',
 'supply',
 'water',
 'in',
 'tankers',
 'Women',
 'took',
 'to',
 'streets',
 'carrying',
 'pots',
 'in',
 'protest',
 'against',
 'erratic',
 'supply',
 'of',
 'drinking',
 'water',
 'at',
 'Kelamagalam',
 'in',
 'Udhanapalli',
 'on',
 'Wednesday',
 '.',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'external',
 'exte

In [16]:
X = pad_sequences(maxlen=train_max_len, sequences=X, padding="post", value=len(vocab)-1)
Y = pad_sequences(maxlen=train_max_len, sequences=Y, padding="post", value=tag2idx["O"])


In [17]:
X[0]

array([ 496, 2058, 4610, 3465,   18, 4319, 5254, 3422, 2062, 5598, 3409,
         91, 4716, 3422, 4321, 1817, 4564, 2086, 1101,  736, 4319, 2634,
        673, 5453,   64, 4169,  736,  172, 1243, 1877,  673, 1967, 1616,
       2255, 4319,  519,   57, 2634, 5716, 1594,  673, 1845, 1598, 2923,
       4453, 5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862,
       5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862,
       5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862,
       5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862,
       5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862,
       5862, 5862, 5862, 5862, 5862, 5862, 5862, 5862], dtype=int32)

In [67]:
embedding_matrix = zeros((len(vocab), 300))
for i,word in enumerate(vocab):
    vec = w2v_dict.get(word)
    if vec is not None:
        embedding_matrix[i] = vec
#e = Embedding(len(vocab), 100, weights=[embedding_matrix], input_length=4, trainable=False)


AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [1]:
a = [1,1,2,3,3,3,3,1,1,2,3,3,5,5,5,5,5]


AttributeError: 'list' object has no attribute 'indexof'